# Sparse Document Retriever - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [4]:
from model_sparse_retriever import TfidfRetriever

In [5]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
import json


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/sparse_retriever.joblib")
        self.model = artifacts["model"]
        self.report_contexts = artifacts["report_contexts"]
    
    def build_result_dataframe(self, sim_contexts_ids,scores):
        sim_contexts = [self.report_contexts[i] for i in sim_contexts_ids[0]]
        df = pd.DataFrame({'doc_id':sim_contexts_ids[0],'score':scores[0],'sim_contexts':sim_contexts})
        df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return df

    def predict(self, X, feature_names, meta=None):

        question = X.copy()[0]
        sim_contexts_ids,scores = self.model(question)
        df = self.build_result_dataframe(sim_contexts_ids,scores)
        df['question'] = [question]*len(scores[0])
        return df.to_numpy() 

Overwriting Model.py


In [6]:
from Model import Model
model = Model()
model.predict(["Qual é o melhor herbicida para erva da ninha ?"],None)

,doc_id,score,sim_contexts
0,842,0.143929,2. INTRODUÇÃO No cultivo moderno de cereais ...
1,681,0.132135,6. CONCLUSÕES Nas condições em que o projet...
2,854,0.113878,4.5 Avaliações: A avaliação de acamamento foi...
3,122,0.087426,4.2 Delineamento experimental e Tratamentos: ...
4,1251,0.086977,3. OBJETIVOS Avaliar a eficácia de doses do ...
...,...,...,...
95,198,0.033807,Tabela 10. Fitotoxicidade visual e produtivida...
96,722,0.033774,Tabela 9. Fitotoxicidade visual e produtividad...
97,691,0.033676,2. INTRODUÇÃO No contexto de plantas daninh...
98,713,0.033650,Tabela 6. Fitotoxicidade visual e produtividad...
